In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Colab Notebooks/DogCat

/content/drive/My Drive/Colab Notebooks/DogCat


In [ ]:
ls

cats_and_dogs_filtered.zip  data/


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-06-08 12:24:47--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

./inception_v3_weig 100%[===================>]  83.84M  63.4MB/s    in 1.3s    

2020-06-08 12:24:49 (63.4 MB/s) - ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
ls

cats_and_dogs_filtered.zip
data/
inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


In [ ]:
# là cái W to đùngchứa các weight ở các lớp , nhưng nó ko biết mạng mình thiết kế ntn . muốn loot vào model thì phải biết cả cấu trúc nữa

In [ ]:
from tensorflow.keras.applications.inception_v3 import  InceptionV3

In [ ]:
local_weight_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
#notop : ko có fully coneted network, ko có cái để phân loại , ko tuong dong se  ko looot dc w

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import  Model



In [ ]:
pre_trained_model = InceptionV3(input_shape=(150,150,3), include_top=False, weights = None) # hien tai chua co weight , dung api de loot w vao 

In [ ]:
pre_trained_model.load_weights(local_weight_file)

In [ ]:
for layers in pre_trained_model.layers:
  layers.trainable = False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 74, 74, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 74, 74, 32)   96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 74, 74, 32)   0           batch_normalization_470[0][0]    
_______________________________________________________________________________________

In [ ]:
# cắt những thứ mình muốn .
last_layer = pre_trained_model.get_layer ('mixed8')

In [ ]:
last_layer.output

<tf.Tensor 'mixed8_5/Identity:0' shape=(None, 3, 3, 1280) dtype=float32>

In [ ]:
# thêm các layer khác  .
from tensorflow.keras.layers import Flatten

In [ ]:
from tensorflow.keras.layers import Dense

In [ ]:
# output , input của nó là tất cả các output đến mix8

x = Flatten()(last_layer.output)

In [ ]:
x= Dense(1024,activation = 'relu')(x)

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
x = Dropout(0.2)(x) #hyperparameter ,khi losss thap , acc cao tranh overfit

In [ ]:
x = Dense(1,activation = 'sigmoid')(x)

In [ ]:
model = Model(pre_trained_model.input,x)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 74, 74, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 74, 74, 32)   96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 74, 74, 32)   0           batch_normalization_470[0][0]    
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer="adam",loss = 'b', metrics = ['acc'])  #op tan dung suc manh dao ham 

In [ ]:
train_folder = '/content/drive/My Drive/Colab Notebooks/DogCat/data/cats_and_dogs_filtered/train'
validate_folder = '/content/drive/My Drive/Colab Notebooks/DogCat/data/cats_and_dogs_filtered/validation'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_generator = ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 40, #xoay
    width_shift_range = 0.2,
    zoom_range = 0.2, 
    horizontal_flip = True
) #cu moi epoch la ta se co nhung anh moi theo setting kia .

In [ ]:
validate_generator = ImageDataGenerator(rescale=1/255.0)

In [ ]:
generated_train_data = train_generator.flow_from_directory(
    train_folder,
    batch_size = 32,
    class_mode= 'binary',
    target_size = (150,150)
)

Found 2000 images belonging to 2 classes.


In [ ]:
generated_valid_data= validate_generator.flow_from_directory(
    validate_folder,
    batch_size = 32,
    class_mode= 'binary',
    target_size = (150,150)

)

Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(generated_train_data,
                              steps_per_epoch = 8,
                              epochs=20,verbose=1,
                              validation_data = (generated_valid_data)
                              )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
8/8 [==============================] - 372s 46s/step - loss: 5.1261 - acc: 0.5547 - val_loss: 1.9277 - val_acc: 0.6980
Epoch 2/20
8/8 [==============================] - 93s 12s/step - loss: 0.9517 - acc: 0.7891 - val_loss: 0.1549 - val_acc: 0.9420
Epoch 3/20
8/8 [==============================] - 86s 11s/step - loss: 0.5186 - acc: 0.8516 - val_loss: 0.1749 - val_acc: 0.9410
Epoch 4/20
8/8 [==============================] - 87s 11s/step - loss: 0.4039 - acc: 0.8672 - val_loss: 0.1560 - val_acc: 0.9500
Epoch 5/20
8/8 [==============================] - 83s 10s/step - loss: 0.2525 - acc: 0.8906 - val_loss: 0.1549 - val_acc: 0.9470
Epoch 6/20
8/8 [==============================] - 77s 10s/step - loss: 0.2113 - acc: 0.9180 - val_loss: 0.1102 - val_acc: 0.9580
Epoch 7/20
8/8 [==============================] - 66s 8s/step - loss: 0.2132 - acc: 0.9180 - val_loss: 0.1051 - val_acc: 0.9600
Epoch 8/20
8/8 [=====